In [4]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
from io import BytesIO

def download_images(search_query, num_images, output_dir):
    # Initialiser le driver Selenium
    options = webdriver.ChromeOptions()
    # Supprimez cette ligne pour debug : options.add_argument("--headless")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Accéder à Google Images
    search_url = f"https://www.google.com/search?tbm=isch&q={search_query}"
    driver.get(search_url)
    time.sleep(2)

    # Vérifier si Google bloque l'accès
    if "Captcha" in driver.page_source or "detected unusual traffic" in driver.page_source:
        print("Google a détecté une activité suspecte. Impossible de continuer.")
        driver.quit()
        return

    print("Page Google Images chargée.")

    # Faire défiler la page pour charger plus d'images
    def scroll_down():
        print("Défilement de la page...")
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)

    # Répéter le défilement jusqu'à obtenir suffisamment d'images
    images = set()
    while len(images) < num_images:
        scroll_down()
        thumbnails = driver.find_elements(By.CSS_SELECTOR, "img")  # Sélecteur générique
        print(f"{len(thumbnails)} miniatures trouvées.")
        for thumbnail in thumbnails:
            src = thumbnail.get_attribute("src") or thumbnail.get_attribute("data-src")
            if src and src.startswith("http"):
                images.add(src)
                print(f"Image collectée : {src}")
                if len(images) >= num_images:
                    break

    # Vérifiez si des images ont été collectées
    if not images:
        print("Aucune image collectée. Terminaison.")
        driver.quit()
        return

    # Créer le répertoire de sortie
    if not os.path.exists(output_dir):
        print(f"Création du dossier : {output_dir}")
        os.makedirs(output_dir)

    # Télécharger les images
    for i, img_url in enumerate(images):
        try:
            print(f"Téléchargement de l'image {i + 1} : {img_url}")
            response = requests.get(img_url, timeout=10)
            img = Image.open(BytesIO(response.content))
            img_path = os.path.join(output_dir, f"{search_query}_{i + 1}.jpg")
            img.save(img_path)
            print(f"Image enregistrée : {img_path}")
        except Exception as e:
            print(f"Erreur lors du téléchargement de l'image {i + 1} : {e}")

    driver.quit()
    print("Téléchargement terminé.")

if __name__ == "__main__":
    query = input("Entrez le terme de recherche : ")
    num = int(input("Entrez le nombre d'images à télécharger : "))
    output_folder = "images"
    download_images(query, num, output_folder)


Page Google Images chargée.
Défilement de la page...
252 miniatures trouvées.
Image collectée : https://fonts.gstatic.com/s/i/productlogos/googleg/v6/24px.svg
Image collectée : https://www.google.com/logos/doodles/2024/seasonal-holidays-2024-6753651837110333.4-s.png
Image collectée : https://fonts.gstatic.com/s/i/productlogos/googleg/v6/24px.svg
Image collectée : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEfGBqV3dKGYknfX40LBvdcDoUKs18MRw4zw&s
Image collectée : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSADDFDM4zGxRzfEUSKuBs6bod5Aruht73B6g&s
Image collectée : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQn-XteR3qM8iJS_wOXOm6yU9PWRI3k2uIX_Q&s
Image collectée : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRZ4vU8OZmrMPqH-Rmc25J_SIjxcxiFGhSHaQ&s
Image collectée : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQkOH4qLsKf0a3RIhYA5SM8wDty4MM4UdMqYg&s
Image collectée : https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSDGZGbgVs0cZxY2CgTWCPeusz

KeyboardInterrupt: 